# Imports 

In [2]:
import numpy as np
np.set_printoptions(suppress=True)
import h5py
import pandas as pd
import time
import glob
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import os
import sys
%matplotlib widget
%load_ext autoreload
%autoreload 2

# --- import functions for computing kinematic variables --- #
sys.path.append('./lib/')
from kinematics import compute_pec_pec_distance_dot

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# load the tracking data

Some kinematics variables can be computed from post-processed data, <br>
but for others it is best to compute from raw data, and do the post-processing afterwards. <br>
So we load both raw and smoothed data. <br>
<br>
We are also going to load arrays which give us fight regions, winner and loser, and th

In [30]:
main_load_folder = '/media/liam/hd1/fighting_data/tracking_results/'

# ----------------------#

interp_polyOrd=1  # the order of the polynomial used for interpolation
interp_limit=5    # the maximum number of frames to interpolate over
savgol_win=9      # the number of frames for the Savitzky-Golay filter
savgol_ord=2      # the polynomial order for the Savitzky-Golay filter
dt=0.01           # the frame rate of the recording

# -----------------------#

loadpaths = glob.glob(os.path.join(main_load_folder, '*results.h5'))
loadpaths.sort()

# parse the exp names
expNames = [path.split('/')[-1][:23] for path in loadpaths]

smooth_trajectories = []
for path in loadpaths:
    with h5py.File(path, 'r') as hf:
        tracks_3D_smooth = hf['tracks_3D_smooth'][:]
    smooth_trajectories.append(tracks_3D_smooth)
    
raw_trajectories = []
for path in loadpaths:
    with h5py.File(path, 'r') as hf:
        tracks_3D_raw = hf['tracks_3D_raw'][:]
    raw_trajectories.append(tracks_3D_raw)
    
    
# -----------------------------#
# load other useful information

    
other_info_loadpath = os.path.join(main_load_folder, 'winners_losers_inconclusive.h5')
with h5py.File(other_info_loadpath, 'r') as hf:
    winner_idxs = hf['winner_idxs'][:]
    loser_idxs = hf['loser_idxs'][:]
    conclusive_winner_loser = hf['conclusive_winner_loser'][:]
    already_established_dominance = hf['already_established_dominance'][:] 

    


# Compute vars for all experiments

In [8]:
from kinematics import compute_pec_pec_distance, compute_thetaW_and_thetaL
from kinematics import compute_signed_pec_z_difference, compute_phi_dot_from_raw_trajectories, compute_coordinate_origin_z

In [9]:
interp_polyOrd=1  # the order of the polynomial used for interpolation
interp_limit=5    # the maximum number of frames to interpolate over
savgol_win=9      # the number of frames for the Savitzky-Golay filter
savgol_ord=2      # the polynomial order for the Savitzky-Golay filter
dt=0.01           # the frame rate of the recording

In [10]:
t0 = time.perf_counter()

exp_dpps = []
exp_tetWs = []
exp_tetLs = []

exp_phi_dot_abs = []
exp_signed_pec_diffs = []
exp_Ozs = []


for ii, expName in enumerate(expNames):
    print(expName)
    
    smooth_traj = smooth_trajectories[ii]
    raw_traj = raw_trajectories[ii]
    winIdx = winner_idxs[ii]
    losIdx = loser_idxs[ii]
    
    dpp_ts = compute_pec_pec_distance(smooth_traj)
    tetW_ts, tetL_ts = compute_thetaW_and_thetaL(smooth_traj, winIdx, losIdx)
    
    
    phi_dot = compute_phi_dot_from_raw_trajectories(raw_traj, winIdx, losIdx, dt=dt,
                                                    interp_max_gap=interp_limit, interp_polyOrd=interp_polyOrd,
                                                    smooth_polyOrd=savgol_ord, smooth_winSize=savgol_win)
    phi_dot_abs_ts = np.abs(phi_dot)
    
    
    signed_pec_z_difference_ts = compute_signed_pec_z_difference(smooth_traj, winIdx, losIdx)
    
    
    Oz_ts = compute_coordinate_origin_z(smooth_traj, winIdx, losIdx)
    
    # ---- record ---- #
    
    exp_dpps.append(dpp_ts)
    exp_tetWs.append(tetW_ts)
    exp_tetLs.append(tetL_ts)
    
    exp_phi_dot_abs.append(phi_dot_abs_ts)
    exp_signed_pec_diffs.append(signed_pec_z_difference_ts)
    exp_Ozs.append(Oz_ts)
    
    
tE = time.perf_counter()
print()
print('finished: {0} s'.format(tE-t0))

FishTank20200127_143538


/home/liam/code/Dynamics_of_dominance/tracking_code/./lib/kinematics.py:194: RuntimeWarning: invalid value encountered in true_divide
  winnerPec_to_loserPec = winnerPec_to_loserPec / row_norms[:, np.newaxis]


FishTank20200129_140656


/home/liam/code/Dynamics_of_dominance/tracking_code/./lib/kinematics.py:194: RuntimeWarning: invalid value encountered in true_divide
  winnerPec_to_loserPec = winnerPec_to_loserPec / row_norms[:, np.newaxis]


FishTank20200130_153857


...

FishTank20200902_160124


/home/liam/code/Dynamics_of_dominance/tracking_code/./lib/kinematics.py:194: RuntimeWarning: invalid value encountered in true_divide
  winnerPec_to_loserPec = winnerPec_to_loserPec / row_norms[:, np.newaxis]


FishTank20200903_160946


/home/liam/code/Dynamics_of_dominance/tracking_code/./lib/kinematics.py:194: RuntimeWarning: invalid value encountered in true_divide
  winnerPec_to_loserPec = winnerPec_to_loserPec / row_norms[:, np.newaxis]



finished: 14.505251819966361 s


# Compute the $D_{PP}$, $\theta_{W}$, $\theta_{L}$  windowed distributions

In [23]:
# ---  time bin params ---- #
window_size=6000   # the window width [frames]
window_step=100    # the step forward in time between windows [frames]

# --- the bins for the variables --- #
dpp_bins = np.linspace(0, 30, 300)
tet_bins = np.linspace(-np.pi, np.pi, 300)

In [24]:
def return_overlapping_windows_for_timeframes(numFrames, window_size=200, window_step=50):
    ''' Given a number of frames, return an 2D array of window start-stop frames.
    '''
    # define, for clarity, the first window
    win0_start = 0
    win0_mid = int(window_size/2)
    win0_end = int(window_size)

    # find numWindows, by adding incrementally and watching the last frame
    last_frame_in_windows = win0_end
    numWindows = 1
    while last_frame_in_windows < (numFrames - window_step):
        numWindows += 1
        last_frame_in_windows = win0_end + (numWindows-1)*window_step

    # now fill-in the windows array of frame indices
    windows = np.zeros((numWindows, 2))
    windows[0, 0] = 0
    windows[0, 1] = win0_end
    for winIdx in range(1, numWindows):
        w0 = winIdx*window_step
        wF = w0 + window_size
        windows[winIdx, 0] = w0
        windows[winIdx, 1] = wF
    return windows.astype(int)




def get_fightbout_rectangle_info(fight_f0, fight_fE, time_windows, spatial_bins):
    ''' Return the (left, bottom, width, height) needed to draw  a rectangle
        representing a fight bout
    '''
    # get the timebin idxs for these frame numbers
    fightbout_start_window = time_windows[time_windows[:,0]>fight_f0][0]
    fight_start_winIdx = np.where(time_windows==fightbout_start_window)[0][0]
    
    # get the stop window,
    # if we hit the end of the experiment, use the last window
    fightbout_stop_window_candidates = time_windows[time_windows[:,0]>fight_fE]
    if len(fightbout_stop_window_candidates) == 0:
        fightbout_stop_window = time_windows[-1]
    else:
        fightbout_stop_window = time_windows[time_windows[:,0]>fight_fE][0]
    fight_stop_winIdx = np.where(time_windows==fightbout_stop_window)[0][0]
    
    # define the shapes we want
    left = fight_start_winIdx
    width = fight_stop_winIdx - left
    bottom = 0
    height = len(spatial_bins)-1
    return left, bottom, width, height


def compute_windowed_distribution_array_from_1D_tseries(tseries, time_windows, spatial_bins):
    ''' Create the x=time, y=var, windowed distribution array for the given tseries.

    -- args --
    tseries: a 1D timeseries
    time_windows: a (N,2) array of start-stop frames i.e the xbins
    spatial_bins: a 1D array of bins in normal format (np.arange(start,stop,step)),
                  i.e. the ybins

    -- returns --
    ts_windowed_data: shape(numBins, numWins)


    -- Note --
    the sns.heatmap outlook indexing is like

    0 1 2 ..
    1
    2
   ...

   But we want y to run the opposite way, like a traditional graph ymin to ymax.
   So what we will do is reverse the histogram values for each window as we record,
   and also swap the ylabels to match everything up.

   '''

    # -- parse some shapes -- #
    numWins = time_windows.shape[0]
    # bins array is edges, so 1 longer than values array, hence the -1
    numBins = spatial_bins.shape[0] - 1
    bin_size = np.abs(spatial_bins[1] - spatial_bins[0])
    bin_min = spatial_bins[0]
    bin_max = spatial_bins[-1]

    # -- compute the distribution --#
    # np.histogram returns bins, which are bin edges, and values in the bins
    # so bins array is 1 longer than values array, hence the -1 in ts_windowed_data shape
    ts_windowed_data = np.zeros((numBins, numWins))
    for winIdx in range(numWins):
        f0, fE = time_windows[winIdx]
        windowed_data = tseries[f0:fE]
        histvals, _ = np.histogram(windowed_data, bins=spatial_bins)
        # now record the histvals backwards, so binmax appears at the top of the figure
        ts_windowed_data[:, winIdx] = histvals[::-1]

    return ts_windowed_data



def make_dpp_tetW_tetL_windowed_dist_figure(expName, dpp_win_dist, tetW_win_dist, tetL_win_dist, dpp_bins, tet_bins, exp_time_wins, conclusive_winner=True,
                                            dpp_vmax=150, tet_vmax=150, num_xticks=10, cmap='Blues', fps=100, use_cbar=False, fight_bout_info_list=None):
    ''' Return the fig, axs values for a figure containing the windowed distributions of dpp,tetW,tetL for each experiment.
    '''
    
    plt.ioff()
    fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10,6), sharex=False)
    fig.suptitle(f"{expName}", fontsize=14)
    
    
    # prepare the drawing of fight-bout regions
    if fight_bout_info_list is not None:
        do_draw_fight_boundaries = True
        # count the number of fights
        numFightBouts = len(fight_bout_info_list)
    

    # ----------------------- dpp ------------------------------#
    ax = axs[0]
    ts_windowed_data = dpp_win_dist
    data_bins = dpp_bins
    panel_title = r'$D_{PP}$ [cm]'

    # some params of the binning
    bin_min = data_bins[0]
    bin_max = data_bins[-1]
    bin_size = np.diff(data_bins)[0]

    # make the heatmap
    heatmap = sns.heatmap(ts_windowed_data, vmax=dpp_vmax, cbar=use_cbar, cmap=cmap, ax=ax);
    if use_cbar == True:
        cbar = heatmap.collections[0].colorbar
        cbar.set_ticks([])
        cbar.set_ticklabels([])

    # prepare the labels
    ax.set_title('')
    ax.set_ylabel(panel_title, fontsize=12)

    # prepare the xticks
    xticks = np.round(np.linspace(0, len(exp_time_wins), num_xticks)).astype(int)
    ax.set_xticks(xticks);
    xtick_labels = (np.linspace(exp_time_wins[0,0], exp_time_wins[-1,1], len(xticks)) / (fps*60)).astype(int)
    ax.set_xticklabels([]);

    # prepare the yticks
    yticks = [0, len(data_bins)]
    ytick_labels = [int(np.round(bin_min)), int(np.round(bin_max))][::-1]
    ax.set_yticks(yticks);
    ax.set_yticklabels(ytick_labels, rotation=0);

    # make frame visible
    for _, spine in heatmap.spines.items():
        spine.set_visible(True)

    # make the frames thicker
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
    ax.xaxis.set_tick_params(width=2, length=8)
    ax.yaxis.set_tick_params(width=2, length=8)
    
    
    # draw the fight-bouts if you want to
    if do_draw_fight_boundaries:
        for boutIdx in range(numFightBouts):
            fight_f0, fight_fE = fight_bout_info_list[boutIdx]
            left, bottom, width, height = get_fightbout_rectangle_info(fight_f0, fight_fE, 
                                                                       exp_time_wins, data_bins)
            rect=mpatches.Rectangle((left,bottom),width,height,
                                    fill=False,
                                    alpha=1,
                                    color="purple")
            ax.add_patch(rect)




    # ----------------------- tet1 ------------------------------#
    ax = axs[1]
    ts_windowed_data = tetW_win_dist
    data_bins = tet_bins
    if conclusive_winner:
        panel_title = r'$\theta_{W}$ [rad]'
    else:
        panel_title = r'$\theta_{?}$ [rad]'

    # some params of the binning
    bin_min = data_bins[0]
    bin_max = data_bins[-1]
    bin_size = np.diff(data_bins)[0]

    # make the heatmap
    heatmap = sns.heatmap(ts_windowed_data, vmax=tet_vmax, cbar=use_cbar, cmap=cmap, ax=ax);
    if use_cbar == True:
        cbar = heatmap.collections[0].colorbar
        cbar.set_ticks([])
        cbar.set_ticklabels([])

    # prepare the labels
    ax.set_title('')
    ax.set_ylabel(panel_title, fontsize=12)

    # prepare the xticks
    xticks = np.round(np.linspace(0, len(exp_time_wins), num_xticks)).astype(int)
    ax.set_xticks(xticks);
    xtick_labels = (np.linspace(exp_time_wins[0,0], exp_time_wins[-1,1], len(xticks)) / (fps*60)).astype(int)
    ax.set_xticklabels([]);

    # prepare the yticks
    yticks = [0, len(data_bins)/2, len(data_bins)]
    ytick_labels = [r'$-\pi$', '0', r'$\pi$'][::-1]
    ax.set_yticks(yticks);
    ax.set_yticklabels(ytick_labels, rotation=0);

    # make frame visible
    for _, spine in heatmap.spines.items():
        spine.set_visible(True)

    # make the frames thicker
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
    ax.xaxis.set_tick_params(width=2, length=8)
    ax.yaxis.set_tick_params(width=2, length=8) 
    
    
    # draw the fight-bouts if you want to
    if do_draw_fight_boundaries:
        for boutIdx in range(numFightBouts):
            fight_f0, fight_fE = fight_bout_info_list[boutIdx]
            left, bottom, width, height = get_fightbout_rectangle_info(fight_f0, fight_fE, 
                                                                       exp_time_wins, data_bins)
            rect=mpatches.Rectangle((left,bottom),width,height,
                                    fill=False,
                                    alpha=1,
                                    color="purple")
            ax.add_patch(rect)




    # ----------------------- tet2 ------------------------------#
    ax = axs[2]
    ts_windowed_data = tetL_win_dist
    data_bins = tet_bins
    if conclusive_winner:
        panel_title = r'$\theta_{L}$ [rad]'
    else:
        panel_title = r'$\theta_{?}$ [rad]'

    # some params of the binning
    bin_min = data_bins[0]
    bin_max = data_bins[-1]
    bin_size = np.diff(data_bins)[0]

    # make the heatmap
    heatmap = sns.heatmap(ts_windowed_data, vmax=tet_vmax, cbar=use_cbar, cmap=cmap, ax=ax);
    if use_cbar == True:
        cbar = heatmap.collections[0].colorbar
        cbar.set_ticks([])
        cbar.set_ticklabels([])

    # prepare the labels
    ax.set_title('')
    ax.set_ylabel(panel_title, fontsize=12)
    ax.set_xlabel('time [min]', fontsize=12)

    # prepare the xticks
    xticks = np.round(np.linspace(0, len(exp_time_wins), num_xticks)).astype(int)
    ax.set_xticks(xticks);
    xtick_labels = (np.linspace(exp_time_wins[0,0], exp_time_wins[-1,1], len(xticks)) / (fps*60)).astype(int)
    ax.set_xticklabels(xtick_labels);

    # prepare the yticks
    yticks = [0, len(data_bins)/2, len(data_bins)]
    ytick_labels = [r'$-\pi$', '0', r'$\pi$'][::-1]
    ax.set_yticks(yticks);
    ax.set_yticklabels(ytick_labels, rotation=0);

    # make frame visible
    for _, spine in heatmap.spines.items():
        spine.set_visible(True)

    # make the frames thicker
    for axis in ['top','bottom','left','right']:
        ax.spines[axis].set_linewidth(2)
    ax.xaxis.set_tick_params(width=2, length=8)
    ax.yaxis.set_tick_params(width=2, length=8) 
    
    
    # draw the fight-bouts if you want to
    if do_draw_fight_boundaries:
        for boutIdx in range(numFightBouts):
            fight_f0, fight_fE = fight_bout_info_list[boutIdx]
            left, bottom, width, height = get_fightbout_rectangle_info(fight_f0, fight_fE, 
                                                                       exp_time_wins, data_bins)
            rect=mpatches.Rectangle((left,bottom),width,height,
                                    fill=False,
                                    alpha=1,
                                    color="purple")
            ax.add_patch(rect)


    # ---------------------------------------------------------------------#

    # wrap up
    fig.tight_layout()
    plt.ion()
    return fig, axs

In [25]:
t0 = time.perf_counter()

exps_time_wins = []
exps_dpp_heatmap = []
exps_tetW_heatmap = []
exps_tetL_heatmap = []



for expIdx in range(len(expNames)):

    # parse expInfo
    expName = expNames[expIdx]
    dpp_ts = exp_dpps[expIdx]
    tetW_ts = exp_tetWs[expIdx]
    tetL_ts = exp_tetLs[expIdx]
    expNfs = dpp_ts.shape[0]
    
    # get the time-windows 
    exp_time_wins = return_overlapping_windows_for_timeframes(expNfs,
                                                              window_size=window_size,
                                                              window_step=window_step)

    # compute the windowed distributions
    dpp_heatmap = compute_windowed_distribution_array_from_1D_tseries(dpp_ts, 
                                                                      exp_time_wins,
                                                                      dpp_bins)
    tetW_heatmap = compute_windowed_distribution_array_from_1D_tseries(tetW_ts, 
                                                                       exp_time_wins,
                                                                       tet_bins)
    tetL_heatmap = compute_windowed_distribution_array_from_1D_tseries(tetL_ts, 
                                                                       exp_time_wins,
                                                                       tet_bins)
    
    # records
    exps_time_wins.append(exp_time_wins)
    exps_dpp_heatmap.append(dpp_heatmap)
    exps_tetW_heatmap.append(tetW_heatmap)
    exps_tetL_heatmap.append(tetL_heatmap)
    

    tE = time.perf_counter()
    print()
    print('finished {0}: {1} s'.format(expName, tE-t0))
    
    
print()
print('------------')
print()
print('finished: {0} s'.format(tE-t0))


finished FishTank20200127_143538: 7.797878697048873 s

finished FishTank20200129_140656: 14.118407862028107 s

finished FishTank20200130_153857: 18.07768521597609 s

finished FishTank20200130_181614: 22.20586526207626 s

finished FishTank20200207_161445: 25.837643664097413 s

finished FishTank20200213_154940: 31.601179210003465 s

finished FishTank20200214_153519: 35.39018562412821 s

finished FishTank20200217_160052: 40.125633660005406 s

finished FishTank20200218_153008: 44.443422441137955 s

finished FishTank20200316_163320: 52.39198087109253 s

finished FishTank20200327_154737: 56.676271511940286 s

finished FishTank20200330_161100: 61.0325827000197 s

finished FishTank20200331_162136: 67.07752054696903 s

finished FishTank20200520_152810: 76.85567037505098 s

finished FishTank20200521_154541: 81.43383074901067 s

finished FishTank20200525_161602: 86.10856007202528 s

finished FishTank20200526_160100: 91.51015354413539 s

finished FishTank20200527_152401: 95.34099095896818 s

fini

# Plot all fight boundaries (raw)

In [26]:
# -----------------------------#
# load other useful information

fight_bout_load_path = os.path.join(main_load_folder, 'fightBouts.h5')
with h5py.File(fight_bout_load_path, 'r') as hf:
    exps_fight_info = hf['exps_fight_info'][:]
    #exps_fight_info = hf['refined_exps_fight_info'][:]
    
# make a dataframe for the fight info
fight_cols = ['expIdxs', 'startIdx', 'stopIdx', 'durations']
fight_bout_df = pd.DataFrame(data=exps_fight_info, columns=fight_cols)

# make a column showing if the experiment ended with a conclusive winner and loser
ends_conclusive_arr = []
df_expIdxs_col = fight_bout_df['expIdxs'].values
for ii, expIdx in enumerate(df_expIdxs_col):
    ends_conclusive_arr.append(conclusive_winner_loser[expIdx])
ends_conclusive_arr = np.array(ends_conclusive_arr)
fight_bout_df['conclusive_winner'] = ends_conclusive_arr


# make a column to show how close each bout is to the end of the experiment
bouts_frames_to_end = []
df_bout_stop_idxs_col = fight_bout_df['stopIdx'].values
df_expIdxs_col = fight_bout_df['expIdxs'].values
for rowIdx,expIdx in enumerate(df_expIdxs_col):
    #print(rowIdx, expIdx)
    row_nfs = smooth_trajectories[expIdx].shape[0]
    row_frame_to_end = row_nfs - df_bout_stop_idxs_col[rowIdx]
    bouts_frames_to_end.append(row_frame_to_end)
bouts_frames_to_end = np.array(bouts_frames_to_end)
fight_bout_df['frames_post_bout'] = bouts_frames_to_end


In [15]:
t0 = time.perf_counter()

fight_info_df = fight_bout_df.copy()


save_folder = '/home/liam/temp/win_dists/exps_fight_info/'
if ~os.path.exists(save_folder):
    os.makedirs(save_folder, exist_ok=True)

# ------------------------------------------------------------------#

for expIdx in range(len(expNames)):

    # parse expInfo
    expName = expNames[expIdx]
    dpp_ts = exp_dpps[expIdx]
    tetW_ts = exp_tetWs[expIdx]
    tetL_ts = exp_tetLs[expIdx]
    expNfs = dpp_ts.shape[0]
    exp_has_conclusive_winner = bool(conclusive_winner_loser[expIdx])
    figsavepath = os.path.join(save_folder, f'{expName}_windists.jpg')
    dpp_heatmap = exps_dpp_heatmap[expIdx]
    tetW_heatmap = exps_tetW_heatmap[expIdx]
    tetL_heatmap = exps_tetL_heatmap[expIdx]
    exp_time_wins = exps_time_wins[expIdx]

    
    # get the fight-bout regions in a list format
    fight_bout_info_arr = fight_info_df[fight_info_df['expIdxs']==expIdx].values
    fight_bout_info_list = [fight_bout_info_arr[i, 1:3] for i in range(fight_bout_info_arr.shape[0])]


    # make the figure
    fig, axs =  make_dpp_tetW_tetL_windowed_dist_figure(expName, 
                                                        dpp_heatmap, tetW_heatmap, tetL_heatmap, 
                                                        dpp_bins, tet_bins, exp_time_wins,
                                                        conclusive_winner=exp_has_conclusive_winner,
                                                        dpp_vmax=150, tet_vmax=150, num_xticks=10, 
                                                        cmap='Blues', fps=100, use_cbar=False, 
                                                        fight_bout_info_list=fight_bout_info_list)
    # save the figure
    fig.savefig(figsavepath, dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

    tE = time.perf_counter()
    print()
    print('finished {0}: {1} s'.format(expName, tE-t0))
    
    
print()
print('------------')
tE = time.perf_counter()
print('finished: {0} s'.format(tE-t0))


finished FishTank20200127_143538: 10.684484839206561 s

finished FishTank20200129_140656: 19.283992247190326 s

finished FishTank20200130_153857: 25.518869223073125 s

finished FishTank20200130_181614: 31.765091869048774 s

finished FishTank20200207_161445: 37.59179147519171 s

finished FishTank20200213_154940: 45.80992733500898 s

finished FishTank20200214_153519: 52.10987548204139 s

finished FishTank20200217_160052: 59.181075636064634 s

finished FishTank20200218_153008: 65.66744540911168 s

finished FishTank20200316_163320: 76.8572670139838 s

finished FishTank20200327_154737: 83.51839919318445 s

finished FishTank20200330_161100: 90.23470200318843 s

finished FishTank20200331_162136: 98.68690185621381 s

finished FishTank20200520_152810: 111.97241205302998 s

finished FishTank20200521_154541: 118.96022941521369 s

finished FishTank20200525_161602: 125.89308046316728 s

finished FishTank20200526_160100: 133.8624869941268 s

finished FishTank20200527_152401: 140.29089471208863 s

f

/tmp/ipykernel_975759/3149694538.py:111: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  fig, axs = plt.subplots(nrows=3, ncols=1, figsize=(10,6), sharex=False)



finished FishTank20200902_160124: 174.6929635622073 s

finished FishTank20200903_160946: 182.99638477014378 s

------------
finished: 182.99689267203212 s


# Plot all fight boundaries (refined)

In [16]:
# -----------------------------#
# load other useful information

fight_bout_load_path = os.path.join(main_load_folder, 'fightBouts.h5')
with h5py.File(fight_bout_load_path, 'r') as hf:
    #exps_fight_info = hf['exps_fight_info'][:]
    exps_fight_info = hf['refined_exps_fight_info'][:]
    
# make a dataframe for the fight info
fight_cols = ['expIdxs', 'startIdx', 'stopIdx', 'durations']
fight_bout_df = pd.DataFrame(data=exps_fight_info, columns=fight_cols)

# make a column showing if the experiment ended with a conclusive winner and loser
ends_conclusive_arr = []
df_expIdxs_col = fight_bout_df['expIdxs'].values
for ii, expIdx in enumerate(df_expIdxs_col):
    ends_conclusive_arr.append(conclusive_winner_loser[expIdx])
ends_conclusive_arr = np.array(ends_conclusive_arr)
fight_bout_df['conclusive_winner'] = ends_conclusive_arr


# make a column to show how close each bout is to the end of the experiment
bouts_frames_to_end = []
df_bout_stop_idxs_col = fight_bout_df['stopIdx'].values
df_expIdxs_col = fight_bout_df['expIdxs'].values
for rowIdx,expIdx in enumerate(df_expIdxs_col):
    #print(rowIdx, expIdx)
    row_nfs = smooth_trajectories[expIdx].shape[0]
    row_frame_to_end = row_nfs - df_bout_stop_idxs_col[rowIdx]
    bouts_frames_to_end.append(row_frame_to_end)
bouts_frames_to_end = np.array(bouts_frames_to_end)
fight_bout_df['frames_post_bout'] = bouts_frames_to_end


In [17]:
t0 = time.perf_counter()

fight_info_df = fight_bout_df.copy()


save_folder = '/home/liam/temp/win_dists/refined_exps_fight_info/'
if ~os.path.exists(save_folder):
    os.makedirs(save_folder, exist_ok=True)

# ------------------------------------------------------------------#

for expIdx in range(len(expNames)):

    # parse expInfo
    expName = expNames[expIdx]
    dpp_ts = exp_dpps[expIdx]
    tetW_ts = exp_tetWs[expIdx]
    tetL_ts = exp_tetLs[expIdx]
    expNfs = dpp_ts.shape[0]
    exp_has_conclusive_winner = bool(conclusive_winner_loser[expIdx])
    figsavepath = os.path.join(save_folder, f'{expName}_windists.jpg')
    dpp_heatmap = exps_dpp_heatmap[expIdx]
    tetW_heatmap = exps_tetW_heatmap[expIdx]
    tetL_heatmap = exps_tetL_heatmap[expIdx]
    exp_time_wins = exps_time_wins[expIdx]

    
    # get the fight-bout regions in a list format
    fight_bout_info_arr = fight_info_df[fight_info_df['expIdxs']==expIdx].values
    fight_bout_info_list = [fight_bout_info_arr[i, 1:3] for i in range(fight_bout_info_arr.shape[0])]


    # make the figure
    fig, axs =  make_dpp_tetW_tetL_windowed_dist_figure(expName, 
                                                        dpp_heatmap, tetW_heatmap, tetL_heatmap, 
                                                        dpp_bins, tet_bins, exp_time_wins,
                                                        conclusive_winner=exp_has_conclusive_winner,
                                                        dpp_vmax=150, tet_vmax=150, num_xticks=10, 
                                                        cmap='Blues', fps=100, use_cbar=False, 
                                                        fight_bout_info_list=fight_bout_info_list)
    # save the figure
    fig.savefig(figsavepath, dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

    tE = time.perf_counter()
    print()
    print('finished {0}: {1} s'.format(expName, tE-t0))
    
    
print()
print('------------')
tE = time.perf_counter()
print('finished: {0} s'.format(tE-t0))


finished FishTank20200127_143538: 11.01671038591303 s

finished FishTank20200129_140656: 19.68322168197483 s

finished FishTank20200130_153857: 25.96991737303324 s

finished FishTank20200130_181614: 32.369182114023715 s

finished FishTank20200207_161445: 38.2106855888851 s

finished FishTank20200213_154940: 46.51822853786871 s

finished FishTank20200214_153519: 53.22851467691362 s

finished FishTank20200217_160052: 60.37351909093559 s

finished FishTank20200218_153008: 66.88579183700494 s

finished FishTank20200316_163320: 77.98870062595233 s

finished FishTank20200327_154737: 84.71431408007629 s

finished FishTank20200330_161100: 91.38331555505283 s

finished FishTank20200331_162136: 99.79388154204935 s

finished FishTank20200520_152810: 113.44056668807752 s

finished FishTank20200521_154541: 120.48651299695484 s

finished FishTank20200525_161602: 127.54133167606778 s

finished FishTank20200526_160100: 135.5854364570696 s

finished FishTank20200527_152401: 141.78647579904646 s

finis

# Plot fig 7 boundaries

In [27]:
# --------   Make the dataFrame from the refined fight-boundaries  ------------------------#

fight_bout_load_path = os.path.join(main_load_folder, 'fightBouts.h5')
with h5py.File(fight_bout_load_path, 'r') as hf:
    exps_fight_info = hf['refined_exps_fight_info'][:]
    
# make a dataframe for the fight info
fight_cols = ['expIdxs', 'startIdx', 'stopIdx', 'durations']
fight_bout_df = pd.DataFrame(data=exps_fight_info, columns=fight_cols)

# make a column showing if the experiment ended with a conclusive winner and loser
ends_conclusive_arr = []
df_expIdxs_col = fight_bout_df['expIdxs'].values
for ii, expIdx in enumerate(df_expIdxs_col):
    ends_conclusive_arr.append(conclusive_winner_loser[expIdx])
ends_conclusive_arr = np.array(ends_conclusive_arr)
fight_bout_df['conclusive_winner'] = ends_conclusive_arr

# make a column to show how close each bout is to the end of the experiment
bouts_frames_to_end = []
df_bout_stop_idxs_col = fight_bout_df['stopIdx'].values
df_expIdxs_col = fight_bout_df['expIdxs'].values
for rowIdx,expIdx in enumerate(df_expIdxs_col):
    #print(rowIdx, expIdx)
    row_nfs = smooth_trajectories[expIdx].shape[0]
    row_frame_to_end = row_nfs - df_bout_stop_idxs_col[rowIdx]
    bouts_frames_to_end.append(row_frame_to_end)
bouts_frames_to_end = np.array(bouts_frames_to_end)
fight_bout_df['frames_post_bout'] = bouts_frames_to_end

In [28]:
# ---------    Apply criteria to fight bouts  ----------------------------------------#
# make a version of the fight-bout dataframe for triggering on starts/stops

# 7 mins, for 2 three-minute segments wihoutout overlap, plus 6000 frames (i.e. 2 30sec frame segments for window overlap past 3 mins)
duration_thresh = 36000 + 6000 

# we don't want fights that are within 3 mins of the end of the experiment
post_Fight_thresh = 18000

# selecting rows based on ending with clear winner/loser, and longer than frame gap
fight_bout_df_for_fight_end_triggered = fight_bout_df.loc[(fight_bout_df['conclusive_winner'] == 1) &
                                                          (fight_bout_df['durations'] >= (duration_thresh)) &
                                                          (fight_bout_df['frames_post_bout'] >= (post_Fight_thresh))
                                                         ]

In [29]:
t0 = time.perf_counter()

fight_info_df = fight_bout_df_for_fight_end_triggered.copy()


save_folder = '/home/liam/temp/win_dists/fig7_bouts/'
if ~os.path.exists(save_folder):
    os.makedirs(save_folder, exist_ok=True)

# ------------------------------------------------------------------#

for expIdx in range(len(expNames)):

    # parse expInfo
    expName = expNames[expIdx]
    dpp_ts = exp_dpps[expIdx]
    tetW_ts = exp_tetWs[expIdx]
    tetL_ts = exp_tetLs[expIdx]
    expNfs = dpp_ts.shape[0]
    exp_has_conclusive_winner = bool(conclusive_winner_loser[expIdx])
    figsavepath = os.path.join(save_folder, f'{expName}_windists.jpg')
    dpp_heatmap = exps_dpp_heatmap[expIdx]
    tetW_heatmap = exps_tetW_heatmap[expIdx]
    tetL_heatmap = exps_tetL_heatmap[expIdx]
    exp_time_wins = exps_time_wins[expIdx]

    
    # get the fight-bout regions in a list format
    fight_bout_info_arr = fight_info_df[fight_info_df['expIdxs']==expIdx].values
    fight_bout_info_list = [fight_bout_info_arr[i, 1:3] for i in range(fight_bout_info_arr.shape[0])]


    # make the figure
    fig, axs =  make_dpp_tetW_tetL_windowed_dist_figure(expName, 
                                                        dpp_heatmap, tetW_heatmap, tetL_heatmap, 
                                                        dpp_bins, tet_bins, exp_time_wins,
                                                        conclusive_winner=exp_has_conclusive_winner,
                                                        dpp_vmax=150, tet_vmax=150, num_xticks=10, 
                                                        cmap='Blues', fps=100, use_cbar=False, 
                                                        fight_bout_info_list=fight_bout_info_list)
    # save the figure
    fig.savefig(figsavepath, dpi=300, transparent=False, bbox_inches='tight', pad_inches=0.1)

    tE = time.perf_counter()
    print()
    print('finished {0}: {1} s'.format(expName, tE-t0))
    
    
print()
print('------------')
tE = time.perf_counter()
print('finished: {0} s'.format(tE-t0))


finished FishTank20200127_143538: 10.676781774032861 s

finished FishTank20200129_140656: 19.455505497055128 s

finished FishTank20200130_153857: 25.641813518013805 s

finished FishTank20200130_181614: 31.95769563713111 s

finished FishTank20200207_161445: 37.853511509951204 s

finished FishTank20200213_154940: 46.14197224006057 s

finished FishTank20200214_153519: 52.24336024513468 s

finished FishTank20200217_160052: 59.308039732975885 s

finished FishTank20200218_153008: 65.78375783702359 s

finished FishTank20200316_163320: 76.82692574104294 s

finished FishTank20200327_154737: 83.60308729996905 s

finished FishTank20200330_161100: 90.2927961179521 s

finished FishTank20200331_162136: 98.6679464140907 s

finished FishTank20200520_152810: 113.05189394997433 s

finished FishTank20200521_154541: 120.02300658798777 s

finished FishTank20200525_161602: 126.94941506697796 s

finished FishTank20200526_160100: 134.92413136409596 s

finished FishTank20200527_152401: 141.04679575492628 s

f